In [1]:
# @title 🛡️ PDF 智慧自動化索引系統 (Enhanced Version)
# @markdown 本工具會自動掃描指定目錄，提取 PDF 摘要並生成 CSV/HTML 報告。

import os
import json
import pandas as pd
from pathlib import Path
from google.colab import drive
from tqdm.notebook import tqdm
import datetime

# 1. 環境適配與依賴安裝
try:
    import fitz  # PyMuPDF
except ImportError:
    print("📦 正在安裝必要元件 (PyMuPDF)...")
    !pip install -q pymupdf
    import fitz

# --- Colab Forms 參數設定區塊 ---
# @markdown ### 📂 路徑與安全設定
source_root = "/content/drive/MyDrive" # @param {type:"string"}
export_folder = "/content/drive/MyDrive/PDF_Index_Project" # @param {type:"string"}
output_name = "pdf_master_index.csv" # @param {type:"string"}

# @markdown ### ⚙️ 執行控制
dry_run = False # @param {type:"boolean"}
# @markdown > **Dry Run 模式**：開啟時僅會列出檔案而不進行 PDF 解析與檔案寫入。
auto_checkpoint = 20 # @param {type:"slider", min:10, max:100, step:10}

class ColabAutomation:
    def __init__(self, source_root, export_folder, output_name):
        self.source = Path(source_root)
        self.export = Path(export_folder)
        self.output_csv = self.export / output_name
        self.checkpoint = self.export / "indexing_progress.json"
        self.html_report = self.export / "dashboard.html"
        self.results = []
        self.processed_paths = set()

    def initialize_env(self):
        """掛載 Drive 並確保輸出目錄存在"""
        if not Path('/content/drive').exists():
            drive.mount('/content/drive')
        self.export.mkdir(parents=True, exist_ok=True)
        print(f"✅ 環境初始化完成。輸出路徑：{self.export}")

    def load_checkpoint(self):
        """載入持久化進度防止重複執行"""
        if self.checkpoint.exists():
            try:
                with open(self.checkpoint, 'r', encoding='utf-8') as f:
                    self.results = json.load(f)
                    self.processed_paths = {item['path'] for item in self.results}
                print(f"🔄 偵測到雲端進度：已跳過 {len(self.processed_paths)} 個檔案。")
            except Exception as e:
                print(f"⚠️ 進度檔損壞，將從頭開始。")

    def preview(self):
        """實作 Dry Run 邏輯"""
        print("🔍 [Dry Run] 正在掃描預計處理的檔案...")
        files = list(self.source.rglob("*.pdf"))
        print(f"📋 預計將處理 {len(files)} 個 PDF 檔案。")
        for i, f in enumerate(files[:5]):
            print(f"   - {f.name}")
        if len(files) > 5: print(f"   ...以及其他 {len(files)-5} 個檔案")
        print("\n💡 若確認無誤，請關閉 dry_run 參數後重新執行。")

    def execute(self):
        """正式執行 PDF 內容提取邏輯"""
        self.load_checkpoint()

        # 取得待處理清單
        all_pdfs = [p for p in self.source.rglob("*.pdf") if str(p) not in self.processed_paths]

        if not all_pdfs:
            print("✅ 所有 PDF 已在索引中，無需更新。")
            return

        pbar = tqdm(all_pdfs, desc="🚀 正在深度分析 PDF", unit="file")

        for i, path in enumerate(pbar):
            pbar.set_postfix_str(f"處理中: {path.name[:15]}")

            info = {
                "filename": path.name,
                "status": "Ready",
                "summary": "",
                "pages": 0,
                "folder": str(path.parent).replace('/content/drive/MyDrive', 'MyDrive'),
                "path": str(path),
                "file_size_bytes": 0, # 新增欄位：檔案大小 (位元組)
                "file_size_mb": 0.0, # 新增欄位：檔案大小 (MB)
                "file_created_date": "", # 新增欄位：檔案創建日期
                "file_modified_date": "" # 新增欄位：檔案修改日期
            }

            try:
                # 取得檔案大小
                file_size_bytes = os.path.getsize(path)
                info["file_size_bytes"] = file_size_bytes
                info["file_size_mb"] = round(file_size_bytes / (1024 * 1024), 2) # 轉換為 MB 並保留兩位小數

                # 取得檔案創建和修改日期
                creation_timestamp = os.path.getctime(path)
                modification_timestamp = os.path.getmtime(path)
                info["file_created_date"] = datetime.datetime.fromtimestamp(creation_timestamp).strftime('%Y-%m-%d %H:%M:%S')
                info["file_modified_date"] = datetime.datetime.fromtimestamp(modification_timestamp).strftime('%Y-%m-%d %H:%M:%S')

                with fitz.open(path) as doc:
                    info["status"] = "✅ 正常" if not doc.is_encrypted else "🔒 加密"
                    info["pages"] = len(doc)
                    if not doc.is_encrypted and len(doc) > 0:
                        text = doc[0].get_text().strip()
                        info["summary"] = " ".join(text.split())[:200]
            except Exception as e:
                info["status"] = f"❌ 錯誤: {type(e).__name__}"

            self.results.append(info)

            # 每隔指定次數儲存進度，防範 IO 異常
            if (i + 1) % auto_checkpoint == 0:
                self._save_checkpoint()

        self._save_final_reports()

    def _save_checkpoint(self):
        with open(self.checkpoint, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, ensure_ascii=False, indent=2)

    def _save_final_reports(self):
        df = pd.DataFrame(self.results)
        # 匯出 CSV (Excel 友善編碼)
        df.to_csv(self.output_csv, index=False, encoding="utf-8-sig")

        # 生成 HTML Dashboard
        html_style = "<style>body{font-family:sans-serif;padding:20px}table{width:100%;border-collapse:collapse}th,td{border:1px solid #ddd;padding:8px;text-align:left}th{background:#3498db;color:white}tr:nth-child(even){background:#f2f2f2}</style>"
        with open(self.html_report, "w", encoding="utf-8") as f:
            f.write(f"<h2>PDF 索引報告</h2>{html_style}")
            f.write(df.to_html(index=False))

        self._save_checkpoint() # 最後存一次確保完整
        print(f"\n✨ 任務完成！報告儲存於: {self.export}")

# --- 啟動邏輯 ---
bot = ColabAutomation(source_root, export_folder, output_name)
bot.initialize_env()

if dry_run:
    bot.preview()
else:
    bot.execute()

📦 正在安裝必要元件 (PyMuPDF)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.0 MB/s eta 0:00:00
Mounted at /content/drive
✅ 環境初始化完成。輸出路徑：/content/drive/MyDrive/PDF_Index_Project


🚀 正在深度分析 PDF:   0%|          | 0/243 [00:00<?, ?file/s]


✨ 任務完成！報告儲存於: /content/drive/MyDrive/PDF_Index_Project


In [ ]:
# @title AI prompt cell

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown,clear_output
from google.colab import ai

dropdown = widgets.Dropdown(
    options=[],
    layout={'width': 'auto'}
)

def update_model_list(new_options):
    dropdown.options = new_options
update_model_list(ai.list_models())

text_input = widgets.Textarea(
    placeholder='Ask me anything....',
    layout={'width': 'auto', 'height': '100px'},
)

button = widgets.Button(
    description='Submit Text',
    disabled=False,
    tooltip='Click to submit the text',
    icon='check'
)

output_area = widgets.Output(
     layout={'width': 'auto', 'max_height': '300px','overflow_y': 'scroll'}
)

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=False)
        accumulated_content = ""
        for new_chunk in ai.generate_text(prompt=text_input.value, model_name=dropdown.value, stream=True):
            if new_chunk is None:
                continue
            accumulated_content += new_chunk
            clear_output(wait=True)
            display(Markdown(accumulated_content))

button.on_click(on_button_clicked)
vbox = widgets.GridBox([dropdown, text_input, button, output_area])

display(HTML("""
<style>
.widget-dropdown select {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
.widget-textarea textarea {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
</style>
"""))
display(vbox)


In [ ]:
# 📦 安裝 Google Sheets API 相關函式庫
# gspread 用於與 Google Sheets API 互動
# gspread-dataframe 用於方便地將 Pandas DataFrame 寫入 Google Sheets
!pip install -q gspread gspread-dataframe

### ⚙️ Google Sheets 身份驗證

為了讓 Colab 能夠存取您的 Google Sheets，您需要進行身份驗證。執行下面的程式碼後，會彈出一個驗證視窗，請依照指示完成授權。**如果您已經掛載了 Google Drive，通常這一步會自動使用現有的認證。**

In [ ]:
import gspread
from google.colab import auth
import pandas as pd
import google.auth # 匯入 google.auth 以取得認證憑證

# 執行 Google 身份驗證
auth.authenticate_user()

# 修正: 從 authenticate_user() 取得的憑證，直接用於 gspread.Client
credentials, project = google.auth.default()
gc = gspread.Client(auth=credentials)

print("✅ Google Sheets 身份驗證成功！")

# 從 Drive 載入之前生成的 CSV 檔案
# 請確保 `export_folder` 和 `output_name` 與上一個儲存格的設定一致
export_folder = "/content/drive/MyDrive/PDF_Index_Project" # @param {type:"string"}
output_name = "pdf_master_index.csv" # @param {type:"string"}
csv_file_path = f"{export_folder}/{output_name}"

try:
    df_to_export = pd.read_csv(csv_file_path, encoding='utf-8-sig')
    print(f"✅ 已成功從 '{csv_file_path}' 載入資料。")
except FileNotFoundError:
    print(f"❌ 錯誤：找不到檔案 '{csv_file_path}'。請確認檔案路徑是否正確，或先執行上方的 PDF 索引程式以生成 CSV 檔案。")
    exit()

# 設定 Google Sheet 名稱
gsheet_name = "PDF Master Index Report" # @param {type:"string"}

# 建立新的 Google Sheet 或開啟現有的 Sheet
try:
    sh = gc.open(gsheet_name)
    print(f"🔄 已開啟現有 Google Sheet: '{gsheet_name}'")
except gspread.exceptions.SpreadsheetNotFound:
    sh = gc.create(gsheet_name)
    print(f"✨ 已建立新的 Google Sheet: '{gsheet_name}'")

# 將資料寫入第一個工作表 (worksheet)
wks = sh.get_worksheet(0) # 獲取第一個工作表

# 清除現有內容 (可選，避免重複資料)
wks.clear()

# 將 DataFrame 寫入 Google Sheet
from gspread_dataframe import set_with_dataframe
set_with_dataframe(wks, df_to_export)

print(f"✅ 資料已成功寫入 Google Sheet: '{gsheet_name}'")
print(f"🔗 您可以在這裡查看 Google Sheet: {sh.url}")